In [1]:
import os
from sklearn import preprocessing
import sys
import numpy as np
import pandas as pd
import pickle
import nltk

In [2]:
df = pd.read_csv(r'D:\sopra_internship\report.csv')

In [3]:
df.head()

,Assignee,text
0,Platform UI Triaged,DG writes: Here is another thing that will ...
1,Platform-Text-Inbox,It would be more efficient if SEF did not cr...
2,Platform UI Triaged,DW (8/15/01 12:24:09 PM) We would like API ...
3,Platform UI Triaged,- open the NOT REQUIREDv view - click on t...
4,JDT-UI-Inbox,- Create two project P1 and P2 - Open the pr...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 2 columns):
Assignee    3998 non-null object
text        3998 non-null object
dtypes: object(2)
memory usage: 62.5+ KB


In [5]:
df['Assignee'].value_counts()

JDT-Text-Inbox                    526
Platform UI Triaged               453
Platform-Text-Inbox               355
JDT-UI-Inbox                      322
platform-cvs-inbox                215
Platform-Compare-Inbox            100
Platform-Resources-Inbox           99
JDT-Core-Inbox                     94
Platform-UI-Inbox                  88
Silenio Quarti                     87
Steve Northover                    83
Project Inbox                      82
Markus Keller                      82
Platform Team Inbox                62
Felipe Heidrich                    57
JDT-Debug-Inbox                    52
Platform-VCM-Inbox                 50
Veronika Irvine                    48
Tod Creasey                        45
Grant Gayed                        43
Platform-Debug-Inbox               38
Platform-Ant-Inbox                 38
pde-build-inbox                    36
cdt-build-inbox@eclipse.org        35
Martin Aeschlimann                 32
platform-runtime-inbox             30
Platform-Upd

In [6]:
le = preprocessing.LabelEncoder()
le.fit(df['Assignee'])

LabelEncoder()

In [7]:
df['Assignee']

0                 Platform UI Triaged
1                 Platform-Text-Inbox
2                 Platform UI Triaged
3                 Platform UI Triaged
4                        JDT-UI-Inbox
5                 Platform UI Triaged
6                 Platform UI Triaged
7                      JDT-Text-Inbox
8                      JDT-Text-Inbox
9                      JDT-Text-Inbox
10                Platform UI Triaged
11                     JDT-Text-Inbox
12                Platform UI Triaged
13                    JDT-Debug-Inbox
14                Platform UI Triaged
15                     JDT-Text-Inbox
16                Platform UI Triaged
17                       JDT-UI-Inbox
18                Platform UI Triaged
19                       JDT-UI-Inbox
20                     Fabio Zadrozny
21                     JDT-Text-Inbox
22             Platform-Compare-Inbox
23                     JDT-Text-Inbox
24                       JDT-UI-Inbox
25                Platform UI Triaged
26          

In [8]:
df['text'].isnull().values.any()

False

In [9]:
raw_text = df['text']

In [10]:
#removing alphanumeric character, decimal digit, whitespace character
processed = raw_text.str.replace(r'[^\w\d\s]', ' ')
processed = raw_text.str.replace(r'\s+', ' ')
processed = raw_text.str.replace(r'^\s+|\s+?$', '')

In [11]:
#converting all to lower case
processed = processed.str.lower()

In [12]:
#stop words
stop_words = nltk.corpus.stopwords.words('english')

In [13]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [14]:
processed = processed.apply(lambda x: ' '.join(
    term for term in x.split() if term not in set(stop_words)))

In [15]:
#stemming using PorterStemmer from NLTK
porter = nltk.PorterStemmer()
processed = processed.apply(lambda x: ' '.join(porter.stem(term) for term in x.split()))

Tokenization is a process of breaking apart the corpus into a vocabulary of unique terms. It has various approaches:
   1) We can tokenize individual terms and generate what's called a bag of words model. You may notice this model has a glaring pitfall: it fails to capture the innate structure of human language. Under this model, the following sentences have the same feature vector although they convey dramatically different meanings.
eg:Does steak taste delicious?
   Steak does taste delicious.
   2) we can tokenize every sequence of n terms called n-grams. For example, tokenizing adjacent pairs of words yields bigrams. The n-gram model preserves word order and can potentially capture more information than the bag of words model.
eg: unigrams and bigrams for "The quick brown fox" are "The", "quick", "brown", "fox", "The quick", "quick brown" and "brown fox".

One statistic called term frequency (tf) tallies the occurrences of each n-gram for every training example. However, some n-grams will show up often while others rarely appear in the overall corpus but show up frequently in certain subsets of messages. Therefore, to emphasize the latter, we'll downweight the term frequency with inverse document frequency (idf), which is calculated by logarithmically scaling the inverse of the fraction of training examples that contain a given term. Combining these two statistics yields the tf-idf statistic:

tf-idf(t,i)=tf(t,i)×idf(t)=tf(t,i)×log(Mmt)
where tf(t,i) is the term frequency for term t in the ith training example, M is the total number of training examples, and mt is the number of training examples that contain the term t.

In [18]:
#tokenization
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
X_ngrams = vectorizer.fit_transform(processed)

C:\Users\sanchitagujral98\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [19]:
X_ngrams.shape

(3998, 175322)

In [ ]:
#splitting into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_ngrams, y_enc, test_size=0.2, random_state=42, stratify=y_enc)
clf = svm.LinearSVC(loss='hinge')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_test, y_pred)